<h1 style="background-color:#FF7733;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🟣 תהליך יצירת נקודות גבול בתהליך לבנק"ל מודרני 🟣
    
</h1>




In [ ]:
print('12. Parcel Border Points In-Process:')

<h1 style="text-align:right;
           font-size:150%; color:#1c6ce6;">
ספריות

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right;
           font-size:150%; color:#1c6ce6;">
    קריאת נתונים  

In [ ]:
print('  12.a Reading inputs')

In [ ]:
RawData_folder = r"\\mapi_shares\MNCDB\צוות מידע\RawData"

today = dt.date.today()
date_name = str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name

del [today, date_name]

<h1 style="text-align:right;font-size:125%">
שכבת נקודת גבול בתהליך

In [ ]:
points_inprocess_cols = ['POINT_NAME', 'ACCURACY', 'MARK_CODE', 'TALAR_NUM', 'TALAR_YEAR', 'USERNAME', 'ENTITY_STATE', 'geometry']

points_inprocess = gpd.read_file(RawData_folder + '/CADSDE/Points_InProcess.gdb',
                                 layer = 'Points_InProcess',
                                 include_fields = points_inprocess_cols)

del points_inprocess_cols

print('         input 1/3 read')

<h1 style="text-align:right;font-size:125%">
שכבת נקודת בקרה

In [ ]:
control_points = gpd.read_file(RawData_folder + '/CADSDE/CONTROL_POINTS.gdb',
                               layer = 'CONTROL_POINTS',
                               include_fields=['geometry'])

del RawData_folder

print('         input 2/3 read')

<h1 style="text-align:right;font-size:125%">
שכבת גבולות תהליכי קדסטר

In [ ]:
cpb_cols = ['CPBUniqueID', 'ProcessName']

cpb = gpd.read_file(product_path,
                    layer = 'CadasterProcessBorders',
                    include_fields = cpb_cols,
                    ignore_geometry = True)

del [cpb_cols, product_path]

print('         input 3/3 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
    עיבוד נתוני נקודות בתהליך  

In [ ]:
print('  12.b Cleaning data')

<h1 style="text-align:right;font-size:100%">
מילוי ערכים חסרים בספרה 0

In [ ]:
points_inprocess[['MARK_CODE', 'ACCURACY', 'TALAR_NUM', 'TALAR_YEAR']] = points_inprocess[['MARK_CODE', 'ACCURACY', 'TALAR_NUM', 'TALAR_YEAR']].fillna(0)
points_inprocess[['MARK_CODE', 'ACCURACY', 'TALAR_NUM', 'TALAR_YEAR']] = points_inprocess[['MARK_CODE', 'ACCURACY', 'TALAR_NUM', 'TALAR_YEAR']].astype(int)

In [ ]:
print('  12.c Data conversion')

<h1 style="text-align:right;font-size:100%">
מיזוג עם שכבת גבולות תהליכי קדסטר מודרנית לקבלת מספר מזהה של התהליך שיצר/עידכן את הנקודה

In [ ]:
points_inprocess['ProcessName'] = points_inprocess['TALAR_NUM'].astype(str) + '/' + points_inprocess['TALAR_YEAR'].astype(str) 

points_inprocess = points_inprocess.merge(right = cpb, on = 'ProcessName', how = 'left')
points_inprocess['CPBUniqueID'] = points_inprocess['CPBUniqueID'].astype('Int64')

points_inprocess.drop(columns = ['TALAR_NUM', 'TALAR_YEAR', 'ProcessName'], inplace = True)
del cpb

<h1 style="text-align:right;font-size:100%">
תיקון ערכים של מעמד הנקודה לערכים 1 או 2

In [ ]:
points_inprocess['ENTITY_STATE'] = points_inprocess['ENTITY_STATE'] + 1
points_inprocess['ENTITY_STATE'].replace(3, 1, inplace=True)
points_inprocess.fillna(1, inplace=True)

points_inprocess.rename(columns = {'ENTITY_STATE' : 'PointStatus'}, inplace = True)
points_inprocess['PointStatus'] = points_inprocess['PointStatus'].astype(int)

<h1 style="text-align:right;font-size:100%">
:מיפוי ערכי סימון נקודת גבול
<h1 style="text-align:right;font-size:100%">
הגדרת מילון מיפוי
<h1 style="text-align:right;font-size:100%">
יצירת שדה חדש עם ערכים חדשים, אם חסר יאוכלס בספרה 0
<h1 style="text-align:right;font-size:100%">
הגדרת השדה כמספר שלם

In [ ]:
MARK_classifier = {0  : 0,
                   1  : 1,
                   2  : 2,
                   3  : 3,
                   4  : 4,
                   5  : 5,
                   6  : 6,
                   7  : 7,
                   8  : 8,
                   9  : 9,
                   10 : 10,
                   11 : 11,
                   12 : 12,
                   13 : 13,
                   14 : 14,
                   15 : 0,
                   20 : 20,
                   None : 0}

points_inprocess['MarkCode'] = points_inprocess['MARK_CODE'].map(MARK_classifier)
points_inprocess['MarkCode'] = points_inprocess['MarkCode'].fillna(0)
points_inprocess['MarkCode'] = points_inprocess['MarkCode'].astype(int)

points_inprocess.drop(columns = 'MARK_CODE', inplace = True)
del MARK_classifier

<h1 style="text-align:right;font-size:100%">
:התניית נקודת בקרה כנקודת גבול
<h1 style="text-align:right;font-size:100%">
איחוד מרחבי עם שכבת נקודות בקרה
<h1 style="text-align:right;font-size:100%">
קבלת שדה שעונה לנקודות גבול שהן גם נקודות בקרה

In [ ]:
control_points['IsControlBorder'] = 1

points_inprocess = gpd.sjoin(left_df = points_inprocess, right_df = control_points,
                             how = 'left',
                             predicate = 'intersects')

points_inprocess['IsControlBorder'] = points_inprocess['IsControlBorder'].fillna(0)
points_inprocess['IsControlBorder'] = points_inprocess['IsControlBorder'].astype(int)

points_inprocess.drop(columns = 'index_right', inplace = True)
del control_points

<h1 style="text-align:right;font-size:100%">
:התניית מקור נקודת גבול
<h1 style="text-align:right;font-size:100%">
הגדרת תנאי כאשר כל הנקודת בעלות סיווג קיים גדול מ-0 ותנאי תלוי בשם משתמש
<h1 style="text-align:right;font-size:100%">
יצירת שדות לשני התנאים שמתארים אם הערכים הרצויים מהתנאי
<h1 style="text-align:right;font-size:100%">
יצירת שדה אחד למקור הנקודה שעונה לשני התנאים

In [ ]:
points_inprocess['ACCURACY'] = points_inprocess['ACCURACY'].fillna(0)

CAD_Sourced_cond = ~points_inprocess['ACCURACY'].isin([0, None, np.nan])
SRV_Sourced_cond = points_inprocess['USERNAME'].isin(['shay', 'yaronl'])


points_inprocess['isCAD'] = np.where(CAD_Sourced_cond, 5, None)
points_inprocess['isSRV'] = np.where(SRV_Sourced_cond, 4, None)

points_inprocess['DataSource'] = points_inprocess['isCAD'].fillna(points_inprocess['isSRV'])
points_inprocess['DataSource'] = points_inprocess['DataSource'].fillna(3)
points_inprocess['DataSource'] = points_inprocess['DataSource'].astype(int)


points_inprocess.drop(columns = ['USERNAME', 'isCAD', 'isSRV'], inplace = True)
del CAD_Sourced_cond, SRV_Sourced_cond

<h1 style="text-align:right;font-size:100%">
הגדרת שדה שם נקודה לפי ערכים קיימים 

In [ ]:
points_inprocess.rename(columns = {'POINT_NAME' : 'PointName'}, inplace = True)

<h1 style="text-align:right;font-size:125%">
:התניית ומיפוי ערכי סיווג לנקודת הגבול
<h1 style="text-align:right;font-size:100%">
ACCURACY מיפוי לפי שדה קיים  

In [ ]:
ACCURACY_classifier = {0    : 24,
                       1    : 1,
                       2    : 13,
                       3    : 13,
                       4    : 13,
                       5    : 13,
                       6    : 24,
                       7    : 24,
                       8    : 24,
                       9    : 24,
                       12   : 12,
                       13   : 13,
                       24   : 24,
                       None : 24}

points_inprocess['Class'] = points_inprocess['ACCURACY'].map(ACCURACY_classifier)
points_inprocess['Class'] = points_inprocess['Class'].fillna(24)
points_inprocess['Class'] = points_inprocess['Class'].astype(int)


points_inprocess.drop(columns = 'ACCURACY', inplace = True)
del ACCURACY_classifier

<h1 style="text-align:right;font-size:100%">
הגדרת כל הנקודות כנקודות דו-ממדיות 

In [ ]:
points_inprocess['Is3D'] = 0
points_inprocess['Is3D'] = points_inprocess['Is3D'].astype(int)

<h1 style="text-align:right;font-size:100%">
יצירת שדה של מספר מזהה לנקודת גבול בתהליך

In [ ]:
coefficient = 20000000
EndID       = len(points_inprocess)+1

points_inprocess['PointUniqueID'] = np.arange(1, EndID) + coefficient
points_inprocess['PointUniqueID'] = points_inprocess['PointUniqueID'].astype(int)

del [coefficient, EndID]

In [ ]:
print('  12.d Defining new Layer')

<h1 style="text-align:right;font-size:100%">
וידוא שהשכבה תישמר ברשת קואורדניטות של ישראל

In [ ]:
points_inprocess.set_geometry('geometry', crs = 2039, inplace = True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות
<h1 style="text-align:right;font-size:100%">
בחירת נתיב 
<h1 style="text-align:right;font-size:100%">
שמירה כשכבה גיאוגרפית

In [ ]:
print('  12.e Exporting results to product folder')

In [ ]:
json_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\BorderPointsInProcess.geojson'

points_inprocess.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\BorderPoints_InProcess-Save as Feature Class.bat"])